In [39]:
import tensorflow as tf
import keras
from keras import Model
from keras import layers
from keras import optimizers
import numpy as np
import matplotlib.pyplot as plt
import SLR_model
import importlib
# from sklearn.model_selection import train_test_split

In [5]:
importlib.reload(SLR_model)

<module 'SLR_model' from 'c:\\Users\\jerry\\Desktop\\hly\\2024-2\\cap\\BN_SLR\\BomNae-SLR-1\\SLR_model.py'>

In [6]:
load_size = 32

In [8]:
# dummy x_raw data
l_raws = []
duration_list = []
for i in range(load_size):
    h_elem = np.zeros(shape=(4,5,3))
    # p_elem = np.zeros(shape=(3,10))
    # comb_elem = [h_elem, h_elem, p_elem]
    x_elem = []
    duration = (np.random.randint(70,91))
    duration_list.append(duration)
    for j in range(duration): # frame size
        x_elem.append(h_elem)
        # x_elem.append(comb_elem)
    l_raws.append(x_elem)
r_raws = []
for i in range(load_size):
    h_elem = np.zeros(shape=(4,5,3))
    # p_elem = np.zeros(shape=(3,10))
    # comb_elem = [h_elem, h_elem, p_elem]
    x_elem = []
    duration = duration_list[i]
    for j in range(duration): # frame size
        x_elem.append(h_elem)
        # x_elem.append(comb_elem)
    r_raws.append(x_elem)
p_raws = []
for i in range(load_size):
    # h_elem = np.zeros(shape=(4,5,3))
    p_elem = np.zeros(shape=(3,10))
    # comb_elem = [h_elem, h_elem, p_elem]
    x_elem = []
    duration = duration_list[i]
    for j in range(duration): # frame size
        # x_elem.append(h_elem)
        x_elem.append(p_elem)
        # x_elem.append(comb_elem)
    p_raws.append(x_elem)
    
loss_weights_raws = []
for i in range(load_size):
    x_elem = []
    duration = duration_list[i]
    for j in range(duration): # frame size
        x_elem.append(1.0)
    loss_weights_raws.append(x_elem)

# dummy y_raw data
# y_raw = np.random.randint(0,1024,load_size)
y_raws = np.zeros(shape=(load_size,), dtype=np.int32)

In [24]:
importlib.reload(SLR_model)

<module 'SLR_model' from 'c:\\Users\\jerry\\Desktop\\hly\\2024-2\\cap\\BN_SLR\\BomNae-SLR-1\\SLR_model.py'>

In [37]:
batch_size = 16
model = SLR_model.get_model()

l_train, each = SLR_model.serialize(l_raws)
r_train, each = SLR_model.serialize(r_raws)
print(len(p_raws), len(loss_weights_raws))
p_train, each, loss_weights = SLR_model.serialize(p_raws, stride=2, loss_weights_list=loss_weights_raws)
y_train_truth = np.repeat(y_raws, each)
y_train_truth = SLR_model.num_arr2bin(y_train_truth, 12)
# check
y_train = (y_train_truth, loss_weights)
x_train = (l_train, r_train, p_train)

print("y_train shape: (", y_train[0].shape,', ', len(y_train[1]), ')')
dataset = SLR_model.convert_to_dataset(x_train, y_train, batch_size)
for i in dataset:
    print(i[1][0].shape)
    break
# hist = model.fit(dataset, epochs=1)


32 32
y_train shape: ( (113, 12) ,  113 )
(16, 12)


---
**keras layer test**

In [102]:
inputs = layers.Input(shape=(32, 3, 10), batch_size=18)
# frame channel feat
d2 = layers.Dense(3, activation='relu')
outputs = layers.TimeDistributed(d2)(inputs)
outputs.shape

TensorShape([18, 32, 3, 3])

In [103]:
outputs = layers.Permute((2, 1, 3))(outputs)
outputs = tf.expand_dims(outputs, axis=4)
outputs.shape

TensorShape([18, 3, 32, 3, 1])

In [104]:
xyz = tf.split(outputs, 3, axis = 1)

In [105]:
xyz

[<KerasTensor: shape=(18, 1, 32, 3, 1) dtype=float32 (created by layer 'tf.split_3')>,
 <KerasTensor: shape=(18, 1, 32, 3, 1) dtype=float32 (created by layer 'tf.split_3')>,
 <KerasTensor: shape=(18, 1, 32, 3, 1) dtype=float32 (created by layer 'tf.split_3')>]

In [147]:
c2 = layers.Conv2D(1, (17,3))
x = c2(xyz[0])
y = c2(xyz[1])
z = c2(xyz[2])
x.shape

TensorShape([18, 1, 16, 1, 1])

In [148]:
x = tf.squeeze(x)
y = tf.squeeze(y)
z = tf.squeeze(z)
x.shape

TensorShape([18, 16])

In [151]:
o = tf.stack([x,y,z], axis=2)
# o = tf.concat([x,y,z], axis = 0)
o.shape

TensorShape([18, 16, 3])

In [43]:
tf.squeeze(outputs)

<KerasTensor: shape=(32, 3, 16, 2) dtype=float32 (created by layer 'tf.compat.v1.squeeze_3')>

- hand model layer test -

In [145]:
inputs = layers.Input(shape=(63, 4, 5, 3), batch_size=32)
# frame, w, h, channel
c21d = Conv2Plus1D(kernel_size=(33,3,3), strides=(2, 1, 1))
outputs = c21d(inputs)
outputs.shape

TensorShape([32, 16, 2, 3, 1])

In [59]:
o2 = layers.Flatten()(outputs)
o2.shape

TensorShape([32, 96])

In [146]:
o2 = outputs
o2 = layers.Reshape((16, o2.shape[2] * o2.shape[3]* o2.shape[4]))(o2)
o2.shape

TensorShape([32, 16, 6])

- combined layer test

In [161]:
l_res = layers.Input(shape=(16, 6), batch_size=32)
r_res = layers.Input(shape=(16, 6), batch_size=32)
p_res = layers.Input(shape=(16, 3), batch_size=32)
fl = layers.Flatten()
lflt =fl(l_res)
rflt = fl(r_res)
pflt = fl(p_res)
ult_flat = tf.concat([lflt, rflt, pflt], axis = 1)
ult_flat.shape

TensorShape([32, 240])

**layer test**

---

**other experiments**

In [138]:
asdf = tf.constant(([0.1, 0.2, 0.3, 0.4],[1, 2, 3, 4], [11,22,33,44]))
# asdf = tf.expand_dims(asdf, axis=0)

asdf.numpy()

array([[ 0.1,  0.2,  0.3,  0.4],
       [ 1. ,  2. ,  3. ,  4. ],
       [11. , 22. , 33. , 44. ]], dtype=float32)

In [140]:
ashpe = asdf.shape
ashpe

TensorShape([3, 4])

In [144]:
asdf = tf.reshape(asdf, (1,12))
ashpe

TensorShape([3, 4])

In [124]:
ababs = tf.split(asdf, 3)
ababs

[<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0.1, 0.2, 0.3, 0.4]], dtype=float32)>,
 <tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[1., 2., 3., 4.]], dtype=float32)>,
 <tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[11., 22., 33., 44.]], dtype=float32)>]

In [125]:
abab = tf.stack([ababs[0],ababs[1],ababs[2]], axis=2)
abab.numpy

<bound method _EagerTensorBase.numpy of <tf.Tensor: shape=(1, 4, 3), dtype=float32, numpy=
array([[[ 0.1,  1. , 11. ],
        [ 0.2,  2. , 22. ],
        [ 0.3,  3. , 33. ],
        [ 0.4,  4. , 44. ]]], dtype=float32)>>

In [99]:
tf.transpose(asdf)

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[ 0.1,  1. , 10. ],
       [ 0.2,  2. , 20. ],
       [ 0.3,  3. , 30. ]], dtype=float32)>

In [92]:
for i,val in enumerate(asdf):
    print(val.numpy())
    print(i)

[1. 1.]
0
[1. 1.]
1
[1. 1.]
2
[1. 1.]
3


---
**Multi input test**

In [3]:
class MultiInputModel(tf.keras.Model):
    def __init__(self):
        super(MultiInputModel, self).__init__()
        
        # Define layers for input 1
        self.dense1 = tf.keras.layers.Dense(2, activation='relu')
        
        # Define layers for input 2
        self.dense3 = tf.keras.layers.Dense(3, activation='relu')

        
        # Common layers for both inputs
        self.concat = tf.keras.layers.Concatenate()
        self.dense_final = tf.keras.layers.Dense(1, activation='sigmoid')
    
    def call(self, inputs):
        # Unpack inputs
        input1, input2 = inputs
        
        # Process input 1
        x1 = self.dense1(input1)
        
        # Process input 2
        x2 = self.dense3(input2)
        
        # Concatenate the processed inputs
        x = self.concat([x1, x2])
        
        # Final dense layer
        output = self.dense_final(x)
        
        return output

In [7]:
model = MultiInputModel()

# Dummy inputs for demonstration (e.g., two inputs with 8 features each)
input1 = np.ones((32, 8))  # Batch size 32, 8 features
input2 = np.zeros((32, 4))  # Batch size 32, 8 features
# input2 = tf.random.normal((32, 4))  # Batch size 32, 8 features

# Forward pass
output = model([input1, input2])

# Print the model summary
model.summary()

# Print the output shape
print(f"Output shape: {output.shape}")

Model: "multi_input_model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             multiple                  18        
                                                                 
 dense_7 (Dense)             multiple                  15        


                                                                 
 concatenate_2 (Concatenate)  multiple                 0         
                                                                 
 dense_8 (Dense)             multiple                  6         
                                                                 
Total params: 39
Trainable params: 39
Non-trainable params: 0
_________________________________________________________________
Output shape: (32, 1)


In [8]:
model.predict([input1, input2])

1/1 [==============================] - 0s 92ms/step


array([[0.8391424],
       [0.8391424],
       [0.8391424],
       [0.8391424],
       [0.8391424],
       [0.8391424],
       [0.8391424],
       [0.8391424],
       [0.8391424],
       [0.8391424],
       [0.8391424],
       [0.8391424],
       [0.8391424],
       [0.8391424],
       [0.8391424],
       [0.8391424],
       [0.8391424],
       [0.8391424],
       [0.8391424],
       [0.8391424],
       [0.8391424],
       [0.8391424],
       [0.8391424],
       [0.8391424],
       [0.8391424],
       [0.8391424],
       [0.8391424],
       [0.8391424],
       [0.8391424],
       [0.8391424],
       [0.8391424],
       [0.8391424]], dtype=float32)